Version 1 of data cleaning and feature engineering.

This version is to ensure that I have a working version of the data which I can use to build and assess classification models.

The general goal is to create a dataset which contains: 

* student biographical information (student table)
* student's course registration information (course and registration tables)
* student's assessment information (assessments table)
  * this needs to be feature engineered to create a single row per student per course
  * as each module has a different number of assessments, this dataset will engineer features which are available for all modules (i.e. average score, proportion of assessments submitted, mean distance from due date, etc.)
* student's VLE information (VLE table)
  * this needs to be feature engineered to create a single row per student per course
  * as each module has a different number of VLE interactions, this dataset will engineer features which are available for all modules (i.e. average number of clicks, proportion of clicks on each resource type, etc.)

Finally, the dataset needs to be sliced in relation to the prediction point:
* students who have withdrawn at the point of prediction need to be excluded - as their outcome is known and will only confuse the model
* more critically, a mechanism to remove assessments, vles, etc. which have 'not 'happened in relation to the prediction point needs to be implemented.  
* this will be the enhanced version - or a future version - of the dataset which will:
  * allow for the prediction of the outcome at any point in the course
  * comparison between time periods


Future versions/questions/scnenarios to explore:

* predict at any point in the course, with information available up to that point
* comparisons between modules - that is, different models per module
* do different models perform better at different points in the course?
* is biographical information useful?
* are there inherent student characteristics which are predictive of outcome?
* are there clusters (unsupervised learning) of students which are predictive of outcome?
* can a model be built which incorporates previous predictions into the model?

BUT - for this assignment, I need to first have something which works and fulfills the assignment specification.



## Data preparation

### Libraries and zip file

In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt


In [2]:
    
# import zip file with csv 
ou_zip = zipfile.ZipFile('../data/anonymisedData.zip') 

# save separate csvs
registrations = pd.read_csv(ou_zip.open('studentRegistration.csv'))
courses = pd.read_csv(ou_zip.open('courses.csv'))
students = pd.read_csv(ou_zip.open('studentInfo.csv'))
student_vle = pd.read_csv(ou_zip.open('studentVle.csv'))
vle = pd.read_csv(ou_zip.open('vle.csv'))
student_assessments = pd.read_csv(ou_zip.open('studentAssessment.csv'))
assessments = pd.read_csv(ou_zip.open('assessments.csv'))

### building dataframe

1. student information as base table
TODO remove rows (students) with missing data in imd_cat

2. add course information from course table - merge on code_module and code_presentation
3. TODO create new features for 'intake', 'year' and 'subject'
   
4. add registration information from registration table - merge on code_module and code_presentation and id_student
TODO do I need to keep 'date_registration'
TODO update missing 'date_unegistration' with max length of course - later step - first use this to subset data (easy way to id registered students)
TODO drop student rows where unregistered before registration date - probably not needed as below will take care of it??
TODO add status column (withdrawn or not withdrawn) at point of prediction:
    WD (true) if final_result == withdrawn and date_unregistration < prediction point
    WD (false) if final_result == withdrawn and date_unregistration > prediction point

    This 'active' or 'withdrawn' status is used to filter whether included in the dataset or not...

    conditions: 

    if final_table['final_result'] == 'Withdrawn' and date_unregistration < prediction point:
        status = withdrawn
    elif final_result == withdrawn and date_unregistration > prediction point:
        status = active
    elif final_table['date_unregistration'].isna():
        status = active
    else:
        status = query

 5. final table = final_table[status = 'active']


6. merge student_asssements with assessments table - merge on code_module and code_presentation and id_assessment
   remove missing scores? - investigate? impute?  could be useful?
   remove cols not going to use (iteration 1, weight, is_banked)

   reduce to only keep those relevant to prediction point - i.e. those which have happened before the prediction point

   assessments = assessemts_combined[date] <= prediction point
   don't use date_submitted as this is student specific; use dates re course assessments
   get number of assessments per course (expected number)
   get number of assessments perstudent row (actual number)
   get sum of scores per student row (sum actual score)
   feature = actual/expected
   feature = sum actual score/actual number * 100
   consider:
   get sum of submission dates (actual submission dates)
   get sum of submission dates (expected submission dates)
   feature = expected submission dates - actual submission dates
   
drop unused, unneeded, columns (type, dates,etc.)
merge with final student table - left join (only active students)

In [5]:
# step 1 merge 'student' with 'course'
final_student = pd.merge(students, courses, on=['code_module', 'code_presentation'], validate='many_to_one')
#final_student.head()

# step 2 merge registrations with final_student
final_student = pd.merge(final_student, registrations, on=['code_module','code_presentation','id_student'], validate="1:1")

final_student.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation_length,date_registration,date_unregistration
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,268,-159.0,NaN
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,268,-53.0,NaN
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,268,-92.0,12.0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,268,-52.0,NaN
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,268,-176.0,NaN


In [ ]:
## to be made into a function?? with a prediction point splitter??

# step 1 merge 'student' with 'course'
final_student = pd.merge(students, courses, on=['code_module', 'code_presentation'], validate='many_to_one')
final_student.head()

# step 2 merge registrations with final_student
final_student = pd.merge(final_student, registrations, on=['code_module','code_presentation','id_student'], validate="1:1")

# step 3 drop imd_band, date_reg nulls and replace NaN for unreg date with the length of the course
final_student.dropna(subset=['date_registration', 'imd_band'], inplace=True)
final_student['date_unregistration'] = final_student['date_unregistration'].fillna(final_student['module_presentation_length'])

# step 4 drop rows where withdrew before started - may want to add cool off buffer 7-14 days? (as earlier)
final_student = final_student[final_student['date_unregistration'] < final_student['date_registration']]

# step 5 split code_presentation into year and month and add subject from code_module
final_student['year'] = final_student['code_presentation'].str[:4].astype(int)
final_student['month'] = final_student['code_presentation'].str[-1].map({'J': 'Oct', 'B': 'Feb'})

# Module subject mapping
code_module_mapping = {
    'AAA': 'SocSci',
    'BBB': 'SocSci',
    'GGG': 'SocSci',
    'CCC': 'Stem',
    'DDD': 'Stem',
    'EEE': 'Stem',
    'FFF': 'Stem'
}
final_student['subject'] = final_student['code_module'].map(code_module_mapping)

#final_student.head()

# step 6 prep assessments - fill missing dates with length of course - 3 days (last week of course)

# merge 'assessments' and 'courses' on 'code_module' and 'code_presentation'
merged_assess = assessments.merge(courses[['code_module', 'code_presentation', 'module_presentation_length']], on=['code_module', 'code_presentation'], how='left')

# value to fill in the missing 'date' values
value_to_fill = merged_assess['module_presentation_length'] - 3

# missing values in 'date' column 
merged_assess['date'] = merged_assess['date'].fillna(value_to_fill)

# update 'assessments' with filled 'date' column
assessments['date'] = merged_assess['date']


# step 7 merge assessments with final_student

## remove cols - is_banked, score = null
